In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

import datetime
import time

import os
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
os.listdir('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/')

['.ipynb_checkpoints',
 'nowcast_data_preprocessing',
 'nowcast_data_preprocessing.ipynb',
 'nowcast_data_preprocessing.zip',
 '가맹점 카드매출액(주,월).csv',
 '가맹점 현금매출액(월).csv',
 '신용카드이용금액(주).csv',
 '실업급여지급건수(월).csv',
 '실업급여지급금액(월).csv',
 '온라인채용 모집인원수(주).csv',
 '전자지급서비스 충전액(주).csv',
 '통신 모바일 인구이동량 통계(2022년 6월 2주차).csv',
 '통신 모바일 인구이동량 통계(2022년 6월 3주차).csv']

In [12]:
card = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/가맹점 카드매출액(주,월).csv')
cash = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/가맹점 현금매출액(월).csv')
fam_card = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/신용카드이용금액(주).csv')
un_num = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/실업급여지급건수(월).csv')
un_amount = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/실업급여지급금액(월).csv')
job_offer = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/온라인채용 모집인원수(주).csv')
fam_charge = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/전자지급서비스 충전액(주).csv')
mobile = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/통신 모바일 인구이동량 통계(2022년 6월 3주차).csv')

In [17]:
all_data = pd.concat([card, cash, fam_card, un_num, un_amount, job_offer, fam_charge], axis=1)
all_data

,Date,전체,전체 4주 이동평균,Date,전체,Date,전체,전체 4주 이동평균,Date,전체,Date,전체,Date,전체,전체 4주 이동평균,Date,전체,전체 4주 이동평균
0,2020-01-10,-0.064023,-6.402267e-02,2020-01-01,0.000000,2020-01-10,-0.049055,-4.905535e-02,2020-01-31,0.000000,2020-01-31,0.000000,2020-01-11,0.005459,0.005459,2020-01-10,-0.034413,-3.441340e-02
1,2020-01-17,-0.007550,-3.578609e-02,2020-02-01,0.021544,2020-01-17,-0.002792,-2.592385e-02,2020-02-29,0.005373,2020-02-29,0.065120,2020-01-18,0.061890,0.033675,2020-01-17,-0.039314,-3.686354e-02
2,2020-01-24,0.066036,-1.845543e-03,2020-03-01,-0.135677,2020-01-24,0.074351,7.500977e-03,2020-03-31,0.176891,2020-03-31,0.230657,2020-01-25,-0.067350,0.000000,2020-01-24,0.093355,6.542630e-03
3,2020-01-31,0.005537,-2.168404e-19,2020-04-01,-0.137917,2020-01-31,-0.022503,-8.673617e-19,2020-04-30,0.258056,2020-04-30,0.355363,2020-02-01,0.212379,0.053095,2020-01-31,-0.019628,2.500000e-09
4,2020-02-07,-0.193122,-3.227472e-02,2020-05-01,-0.056387,2020-02-07,-0.183668,-3.365319e-02,2020-05-31,0.242322,2020-05-31,0.379569,2020-02-08,0.133046,0.084991,2020-02-07,0.001570,8.995767e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2022-05-13,0.205175,1.772388e-01,NaN,NaN,2022-05-13,0.211520,1.648906e-01,NaN,NaN,NaN,NaN,2022-05-14,0.043726,0.257965,2022-05-27,0.737057,7.769838e-01
123,2022-05-20,0.118175,1.876140e-01,NaN,NaN,2022-05-20,0.116628,1.767616e-01,NaN,NaN,NaN,NaN,2022-05-21,0.483005,0.294000,2022-06-03,0.785399,7.551523e-01
124,2022-05-27,0.293399,1.654167e-01,NaN,NaN,2022-05-27,0.277204,1.607548e-01,NaN,NaN,NaN,NaN,2022-05-28,0.072052,0.232318,2022-06-10,0.680380,7.063081e-01
125,2022-06-03,0.189387,2.015339e-01,NaN,NaN,2022-06-03,0.135544,1.852239e-01,NaN,NaN,NaN,NaN,2022-06-04,0.102891,0.175418,2022-06-17,0.700501,7.258345e-01


In [18]:
input_df_date(all_data, '2020-01', '2020-11')

AttributeError: 'DataFrame' object has no attribute 'str'

### preprocessing function

In [16]:
## all data preprocessing (monthly variation)
def input_df_date(df, start_year_month, end_year_month):
    date_list = []
    month_value_list = []

    try:
        df['year_month'] = df['Date'].str.slice(stop=7)
        
    except:
        df = df.T
        df.columns = df.iloc[0]
        df = df.drop(df.index[0]).reset_index(drop=True)
        df = df.drop(['관내', '관외'], axis=1)

        df['year_month'] = df['전국 총계'].str.slice(stop=7)
        df['year_month'] = [df['year_month'][i].replace('.', '-') for i in range(len(df['year_month']))]
        df['전체'] = [int(df['합계'][i].replace(',', '')) for i in range(len(df['year_month']))]

        df.drop(['전국 총계', '합계'], axis=1, inplace=True)
        

    for date in df.year_month.unique():
        four_weeks = df[df['year_month']==date].reset_index(drop=True)

        week_value_list = []
        week_value_list = four_weeks['전체'].to_list()

        month_value = np.log(sum([a for a in np.exp(week_value_list)]))
        month_value_list.append(month_value)
        date_list.append(date)


    df_month = pd.DataFrame(columns=['date', 'month_value'])
    df_month['date'] = date_list
    df_month['month_value'] = month_value_list
    
    df_month = df_month.set_index('date')

    term = df_month.reset_index()
    start = term.loc[term['date']==start_year_month].index.values
    end = term.loc[term['date']==end_year_month].index.values
    result = term.iloc[int(start):int(end)+1]
    
    result = result.set_index('date')
    
    return result

In [7]:
    mobile = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/통신 모바일 인구이동량 통계(2022년 6월 3주차).csv')

    date_list = []
    month_value_list = []

    try:
        mobile['year_month'] = mobile['Date'].str.slice(stop=7)
        
    except:
        mobile = mobile.T
        mobile.columns = mobile.iloc[0]
        mobile = mobile.drop(mobile.index[0]).reset_index(drop=True)
        mobile = mobile.drop(['관내', '관외'], axis=1)

        mobile['year_month'] = mobile['전국 총계'].str.slice(stop=7)
        mobile['year_month'] = [mobile['year_month'][i].replace('.', '-') for i in range(len(mobile['year_month']))]
        mobile['전체'] = [int(mobile['합계'][i].replace(',', '')) for i in range(len(mobile['year_month']))]

        mobile.drop(['전국 총계', '합계'], axis=1, inplace=True)        

    for date in mobile.year_month.unique():
        four_weeks = mobile[mobile['year_month']==date].reset_index(drop=True)

        week_value_list = []
        week_value_list = four_weeks['전체'].to_list()
        
        
        month_value = np.exp(week_value_list)

#         month_value = np.log(sum([a for a in np.exp(week_value_list)]))
#         month_value_list.append(month_value)
#         date_list.append(date)

#     mobile_month = pd.DataFrame(columns=['date', 'month_value'])
#     mobile_month['date'] = date_list
#     mobile_month['month_value'] = month_value_list
    
#     mobile_month = mobile_month.set_index('date')
    
#     term = mobile_month.reset_index()
#     start = term.loc[term['date']=='2020-01'].index.values
#     end = term.loc[term['date']=='2020-11'].index.values
#     result = term.iloc[int(start):int(end)+1]
    
#     result = result.set_index('date')
    
#     result

In [8]:
np.exp(week_value_list)

array([inf, inf, inf])

### function result

In [18]:
input_df_date(un_num, '2020-01', '2020-11') ## 모바일인구이동량 통계를 제외하고는 모두 아래 형태로 출력이 됩니다.

,month_value
date,
2020-01,0.000000
2020-02,0.005373
2020-03,0.176891
2020-04,0.258056
2020-05,0.242322
2020-06,0.356605
2020-07,0.444268
2020-08,0.308331
2020-09,0.430579


In [37]:
week_to_month(mobile)

,month_value
date,
2019-01,inf
2019-02,inf
2019-03,inf
2019-04,inf
2019-05,inf
2019-06,inf
2019-07,inf
2019-08,inf
2019-09,inf


In [95]:
## 1월달 변동량으로 테스트 (수동계산)
test = card.iloc[0:4,1].to_list()
test

[-0.06402267, -0.00754951, 0.06603555, 0.00553663]

In [94]:
## exp
exp_test = np.exp(test)
exp_test

array([inf, inf, inf, inf])

In [9]:
np.exp(-0.06402267)

0.9379837351984223

In [10]:
## sum
sum_exp_test = np.sum(exp_test)
sum_exp_test

4.004279329932788

In [11]:
round(0.93798374+0.99247892+1.06826469+1.00555199, 10)

4.00427934

In [12]:
## log
log_sum_exp_test = np.log(sum_exp_test)
log_sum_exp_test

1.3873636217401453